In [182]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_list, struct
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
import codecs
spark = SparkSession.builder.getOrCreate()

In [183]:
df0 = spark.read.json('sample_10e3.json')

In [184]:
df.show()
df.count()

+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|travel_time|     unplug_hourTime|       user_day_code|user_type|zip_code|
+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|{5cf83b752f3843a0...|       0|         21|            66|            8|              90|        219|{2019-06-01T00:00...|e4d55deb9ac172a8d...|        1|        |
|{5cf83b762f3843a0...|       4|         19|           136|           19|              71|        359|{2019-06-01T00:00...|8a0c4123e924a50a9...|        1|   28039|
|{5cf83b762f3843a0...|       4|         17|            38|            7|              39|        375|{2019-06-01T00:00...|a6a9c1f74a6849600...|        1|   28013|
|{5cf83b762f3843a0...|

1000

In [185]:
#df = df.drop('_id','unplug_hourTime','idplug_base','idunplug_base','track','idplug_station','idunplug_station','idunplug_station','user_day_code','user_type','zip_code')
df = df0
df = df.select('ageRange','travel_time')
df.show()
# to fit df in page and see more clearly the results

+--------+-----------+
|ageRange|travel_time|
+--------+-----------+
|       0|        219|
|       4|        359|
|       4|        375|
|       5|        264|
|       4|        367|
|       5|        174|
|       0|        308|
|       4|        462|
|       0|        482|
|       4|        480|
|       5|        546|
|       0|        519|
|       4|        374|
|       4|        188|
|       4|        133|
|       0|        514|
|       4|        517|
|       4|        525|
|       0|        351|
|       0|        483|
+--------+-----------+
only showing top 20 rows



In [186]:
# APARTADO 1: Tiempo que una persona está en la bici en función de su grupo de edad.

# intentos fallidos con lo de carmona
#df.groupBy('ageRange').agg({'travel_time':'sum'}).show()
#unos = udf(lambda x: 1, IntegerType())
#df1 = df
#df1.select(unos('ageRange'))
#df = df.withColumn('contador', unos(df1['ageRange']))

df.show()
media = udf (lambda lista: sum(lista)/len(lista))
df = df.groupBy('ageRange').agg(collect_list('travel_time').alias('lista_tiempos')).sort('ageRange')
df.show()
df.withColumn('media_tiempos', media(df['lista_tiempos'])).show()

+--------+-----------+
|ageRange|travel_time|
+--------+-----------+
|       0|        219|
|       4|        359|
|       4|        375|
|       5|        264|
|       4|        367|
|       5|        174|
|       0|        308|
|       4|        462|
|       0|        482|
|       4|        480|
|       5|        546|
|       0|        519|
|       4|        374|
|       4|        188|
|       4|        133|
|       0|        514|
|       4|        517|
|       4|        525|
|       0|        351|
|       0|        483|
+--------+-----------+
only showing top 20 rows

+--------+--------------------+
|ageRange|       lista_tiempos|
+--------+--------------------+
|       0|[219, 308, 482, 5...|
|       1|[1146, 1150, 1151...|
|       2|[466, 404, 210, 1...|
|       3|[477, 574, 435, 7...|
|       4|[359, 375, 367, 4...|
|       5|[264, 174, 546, 6...|
|       6|[1036, 1108, 1111...|
+--------+--------------------+

+--------+--------------------+------------------+
|ageRange|       l

In [205]:
# APARTADO 2: Barrios con más salidas y llegadas. 
df = df0
df = df.select('zip_code','idplug_station','idunplug_station').filter(col('zip_code')!= '')
df.show()
# sería con esto, pero hay q mirar q estaciones estan en el mismo barrio....
# zip code es el codigo postal del usuario q realiza el movimiento.....

+--------+--------------+----------------+
|zip_code|idplug_station|idunplug_station|
+--------+--------------+----------------+
|   28039|           136|              71|
|   28013|            38|              39|
|   28009|            90|              66|
|   28006|           166|             152|
|   28907|            53|              55|
|   28003|           169|             153|
|   28004|           133|              85|
|   28007|            83|              31|
|   39316|            13|             118|
|   28012|            10|              27|
|   28045|            74|              90|
|   28008|            83|              67|
|   28019|            83|              67|
|   28028|           171|              77|
|   28029|           151|              99|
|   28006|             5|             156|
|   28006|             5|             156|
|   28010|             5|             156|
|   28026|           135|              31|
|   28007|            94|             130|
+--------+-

In [271]:
# APARTADO 3. Horas en las que hay más desenganches de bici en función del barrio y del tiempo que viajan.
# lo hago en funcion del barrio, en funcion del tiempo es analogo


# problema: no sé convertir la hora a un entero

import struct
df = df0
df = df.select('zip_code','unplug_hourTime').filter(col('zip_code')!= '')
hora = udf(lambda x: int(str(x)[23]) if str(x)[22]== '0' else int(str(x)[22]+str(x)[23])) # por lo visto no salen enteros
df = df.withColumn('hora', hora(df['unplug_hourTime'])).drop('unplug_hourTime')
#print(df.dtypes)
df.show()
df = df.groupBy('zip_code').agg(collect_list('hora').alias('lista_horas')).sort('zip_code')
df.show()
media = udf (lambda lista: sum(lista)/len(lista))
df.withColumn('hora_media', media(df['lista_horas'])).show()


+--------+----+
|zip_code|hora|
+--------+----+
|   28039|   0|
|   28013|   0|
|   28009|   0|
|   28006|   0|
|   28907|   0|
|   28003|   0|
|   28004|   0|
|   28007|   0|
|   39316|   0|
|   28012|   0|
|   28045|   0|
|   28008|   0|
|   28019|   0|
|   28028|   0|
|   28029|   0|
|   28006|   0|
|   28006|   0|
|   28010|   0|
|   28026|   0|
|   28007|   0|
+--------+----+
only showing top 20 rows

+--------+--------------------+
|zip_code|         lista_horas|
+--------+--------------------+
|   06800|                 [0]|
|    2008|              [0, 1]|
|   28001|[0, 0, 0, 0, 0, 1...|
|   28002|  [0, 0, 0, 0, 0, 1]|
|   28003|[0, 0, 0, 0, 0, 0...|
|   28004|[0, 0, 0, 0, 0, 0...|
|   28005|[0, 0, 0, 0, 0, 0...|
|   28006|[0, 0, 0, 0, 0, 0...|
|   28007|[0, 0, 0, 0, 0, 0...|
|   28008|[0, 0, 0, 0, 0, 0...|
|   28009|[0, 0, 0, 0, 0, 0...|
|   28010|[0, 0, 0, 0, 0, 0...|
|   28011|              [0, 1]|
|   28012|[0, 0, 0, 0, 0, 0...|
|   28013|[0, 0, 0, 0, 0, 0...|
|   28014|[0, 

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "<ipython-input-271-e3f6d3c3f177>", line 12, in <lambda>
TypeError: unsupported operand type(s) for +: 'int' and 'str'


In [211]:
#3
df_trips = df1.groupBy('user_day_code').count()
df_trips.show()

AttributeError: 'NoneType' object has no attribute 'groupBy'

In [ ]:
df_trips.filter(df_trips['count']==1).show()

In [105]:
#lo mismo en una línea
from pyspark.sql.functions import col
df1.groupBy('user_day_code').count().alias('trips').filter(col('trips.count')==1).show()

AnalysisException: cannot resolve 'user_day_code' given input columns: [ageRange, contador, travel_time];
'Aggregate ['user_day_code], ['user_day_code, count(1) AS count#2266L]
+- Project [ageRange#972L, travel_time#977L, <lambda>(ageRange#972L) AS contador#2192]
   +- Project [ageRange#972L, travel_time#977L, <lambda>(ageRange#972L) AS contador#2181]
      +- Project [ageRange#972L, travel_time#977L, <lambda>(ageRange#972L) AS contador#2144]
         +- Project [ageRange#972L, travel_time#977L, <lambda>(ageRange#972L) AS contador#2080]
            +- Project [ageRange#972L, travel_time#977L, <lambda>(ageRange#972L) AS contador#2043]
               +- Project [ageRange#972L, travel_time#977L, <lambda>(ageRange#972L) AS contador#2035]
                  +- Project [ageRange#972L, travel_time#977L, <lambda>(ageRange#972L) AS contador#1996]
                     +- Project [ageRange#972L, travel_time#977L, <lambda>(ageRange#972L) AS contador#1985]
                        +- Project [ageRang

In [10]:
#4. Histograma de viajes
df1.groupBy('user_day_code').count().groupBy('count').count().show()

+-----+-----+
|count|count|
+-----+-----+
|    1|  873|
|    3|    1|
|    2|   39|
+-----+-----+



In [11]:
df1.groupBy('user_day_code').count().withColumnRenamed('count','trip_number').\
    groupBy('trip_number').count().show()

+-----------+-----+
|trip_number|count|
+-----------+-----+
|          1|  873|
|          3|    1|
|          2|   39|
+-----------+-----+



In [12]:
#5. Return trip. 

#En las celdas anteriores, vemos que al agrupar perdemos la información.  
#se echa de menos el poder trabajar con los datos agregados.


In [31]:
from pyspark.sql.functions import collect_list, struct
dfg = df.groupBy('user_day_code').agg(collect_list(struct('idunplug_station','idplug_station')))
dfg.show()

+--------------------+------------------------------------------------------+
|       user_day_code|collect_list(struct(idunplug_station, idplug_station))|
+--------------------+------------------------------------------------------+
|003a61da2de58f96b...|                                             [{9, 16}]|
|004b59e8e915fea89...|                                          [{171, 106}]|
|00c762bdd28e33ad6...|                                            [{19, 90}]|
|00e51402a96f66595...|                                            [{26, 79}]|
|01232b2ca4316d5a8...|                                           [{12, 174}]|
|015d7f04c3112bc80...|                                           [{174, 31}]|
|016def195d7b6e1f9...|                                           [{156, 63}]|
|0171f8de1acd7d8c7...|                                            [{62, 82}]|
|017cc6f6baa1d23a0...|                                            [{88, 54}]|
|019909d2442ab142c...|                                          

In [14]:
dfg = df.groupBy('user_day_code').\
    agg(collect_list(struct('idunplug_station','idplug_station')).alias('result'))
dfg.show()

+--------------------+-------------------+
|       user_day_code|             result|
+--------------------+-------------------+
|003a61da2de58f96b...|          [{9, 16}]|
|004b59e8e915fea89...|       [{171, 106}]|
|00c762bdd28e33ad6...|         [{19, 90}]|
|00e51402a96f66595...|         [{26, 79}]|
|01232b2ca4316d5a8...|        [{12, 174}]|
|015d7f04c3112bc80...|        [{174, 31}]|
|016def195d7b6e1f9...|        [{156, 63}]|
|0171f8de1acd7d8c7...|         [{62, 82}]|
|017cc6f6baa1d23a0...|         [{88, 54}]|
|019909d2442ab142c...|         [{90, 82}]|
|01afe1971a4a4fa91...|         [{76, 81}]|
|01bf069d0732b2b01...|         [{42, 84}]|
|01ce39793f7b766e3...|       [{169, 149}]|
|0247f880037914475...|         [{84, 57}]|
|02cb8492c91c06764...|        [{169, 26}]|
|02f00ebaf4701793d...|         [{90, 82}]|
|0382218c6e10772dc...|[{45, 71}, {71, 8}]|
|039ae0fa68bc44bf0...|       [{111, 155}]|
|03d3c2c2df2d08e0f...|        [{135, 51}]|
|0444abbe992567898...|         [{38, 52}]|
+----------

In [15]:
#aplicar la función de abajo...para calcular la longitud... ver si podemos hacer más cosas, como ver 
# que es un round trip

In [19]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

trips_len = udf(lambda x: len(x), IntegerType())
dfg.select(trips_len('user_day_code')).show()

+-----------------------+
|<lambda>(user_day_code)|
+-----------------------+
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
|                     64|
+-----------------------+
only showing top 20 rows



In [13]:
dfg2 = dfg.withColumn('len_udc', trips_len(dfg['user_day_code']))
dfg2.show()

+--------------------+-------------------+-------+
|       user_day_code|             result|len_udc|
+--------------------+-------------------+-------+
|003a61da2de58f96b...|          [{9, 16}]|     64|
|004b59e8e915fea89...|       [{171, 106}]|     64|
|00c762bdd28e33ad6...|         [{19, 90}]|     64|
|00e51402a96f66595...|         [{26, 79}]|     64|
|01232b2ca4316d5a8...|        [{12, 174}]|     64|
|015d7f04c3112bc80...|        [{174, 31}]|     64|
|016def195d7b6e1f9...|        [{156, 63}]|     64|
|0171f8de1acd7d8c7...|         [{62, 82}]|     64|
|017cc6f6baa1d23a0...|         [{88, 54}]|     64|
|019909d2442ab142c...|         [{90, 82}]|     64|
|01afe1971a4a4fa91...|         [{76, 81}]|     64|
|01bf069d0732b2b01...|         [{42, 84}]|     64|
|01ce39793f7b766e3...|       [{169, 149}]|     64|
|0247f880037914475...|         [{84, 57}]|     64|
|02cb8492c91c06764...|        [{169, 26}]|     64|
|02f00ebaf4701793d...|         [{90, 82}]|     64|
|0382218c6e10772dc...|[{45, 71}

In [34]:
from pyspark.sql.types import BooleanType

def round_trip(x):
    if len(x) != 2:
        return False
    else:
        return x[0][1] == x[1][0] and x[0][0] == x[1][1]

round_trip_udf = udf(lambda x: round_trip(x), BooleanType())

dfg.withColumn('round_trip', round_trip_udf(dfg['result'])).alias('dfgrt').filter(col('dfgrt.round_trip')).show()


+--------------------+--------------------+----------+
|       user_day_code|              result|round_trip|
+--------------------+--------------------+----------+
|13b9c8b5ffecc4d2a...|[{125, 6}, {6, 125}]|      true|
|6fb07b5e128af9a6b...|[{57, 19}, {19, 57}]|      true|
|a1e5829f6bb2030d7...|[{129, 57}, {57, ...|      true|
|b10c476e28ef97b3f...|[{96, 114}, {114,...|      true|
|bcef457f1234cc899...|[{96, 114}, {114,...|      true|
|f7f4cd3b9b0f8c51b...|[{62, 169}, {169,...|      true|
+--------------------+--------------------+----------+

